# SI 618 Fall 2021 Homework 3 (100 points)

## Part 1 (75 points) 

The provided ‘movie_actors_data.txt’ file contains a JSON string on each line. For example, the first line is: 

>{"rating": 9.3, "genres": ["Crime", "Drama"], "rated": "R", "filming_locations": "Ashland, Ohio, USA", "language": ["English"], "title": "The Shawshank Redemption", "runtime": ["142 min"], "poster": "http://img3.douban.com/lpic/s1311361.jpg", "imdb_url": "http://www.imdb.com/title/tt0111161/", "writers": ["Stephen King", "Frank Darabont"], "imdb_id": "tt0111161", "directors": ["Frank Darabont"], "rating_count": 894012, "actors": ["Tim Robbins", "Morgan Freeman", "Bob Gunton", "William Sadler", "Clancy Brown", "Gil Bellows", "Mark Rolston", "James Whitmore", "Jeffrey DeMunn", "Larry Brandenburg", "Neil Giuntoli", "Brian Libby", "David Proval", "Joseph Ragno", "Jude Ciccolella"], "plot_simple": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.", "year": 1994, "country": ["USA"], "type": "M", "release_date": 19941014, "also_known_as": ["Die Verurteilten"]}

The fields we are interested in are imdb_id , title , rating, genres, actors, and year. You will parse the JSON strings, and load the data into three tables in SQLite, and then write SQL queries to retrieve the data specified. 

You will create three tables: 
* The “movie_genre” table, which has two columns: imdb_id and genre. A movie typically has multiple genres, and in this case, there should be one row for each genre. If some movie does not have any genre, ignore that movie.  

* The “movies” table, which has four columns: imdb_id, title, year, rating, country  

* The “movie_actor” table, which has two columns imdb_id and actor. A movie typically has  
multiple actors, and in this case, there should be one row for each actor. 

**When you run your Python code, it should print out EXACTLY the output provided.**

#### Q1. (10 points) Parse input file to get needed data for the three tables and load them into appropriate Python data structure. 

In [1]:
import json
import sqlite3

with open("movie_actors_data.txt", "r") as f:
    movie_actors_data = [json.loads(x.replace("\n", "")) for x in f.readlines()]
movie_actors_data[0]

{'rating': 9.3,
 'genres': ['Crime', 'Drama'],
 'rated': 'R',
 'filming_locations': 'Ashland, Ohio, USA',
 'language': ['English'],
 'title': 'The Shawshank Redemption',
 'runtime': ['142 min'],
 'poster': 'http://img3.douban.com/lpic/s1311361.jpg',
 'imdb_url': 'http://www.imdb.com/title/tt0111161/',
 'writers': ['Stephen King', 'Frank Darabont'],
 'imdb_id': 'tt0111161',
 'directors': ['Frank Darabont'],
 'rating_count': 894012,
 'actors': ['Tim Robbins',
  'Morgan Freeman',
  'Bob Gunton',
  'William Sadler',
  'Clancy Brown',
  'Gil Bellows',
  'Mark Rolston',
  'James Whitmore',
  'Jeffrey DeMunn',
  'Larry Brandenburg',
  'Neil Giuntoli',
  'Brian Libby',
  'David Proval',
  'Joseph Ragno',
  'Jude Ciccolella'],
 'plot_simple': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'year': 1994,
 'country': ['USA'],
 'type': 'M',
 'release_date': 19941014,
 'also_known_as': ['Die Verurteilten']}

#### Q2. (5 points) Create the movie_genre table and load data into it.

In [2]:
movie = sqlite3.connect('movie.db')
movie_c = movie.cursor()
movie_c.execute(
    'CREATE TABLE movie_genre (imdb_id text, genre text)'
)
movie_data_temp = []
for m in movie_actors_data:
   movie_data_temp += [
       (m["imdb_id"], g) for g in m["genres"]
   ] 
movie_c.executemany(
    'INSERT INTO movie_genre VALUES (?, ?)',
    movie_data_temp
    )
movie.commit()

In [8]:
len(movie_data_temp)

699

#### Q3. (5 points) Create the movies table and load data into it.

In [3]:
movie_c.execute(
    'CREATE TABLE movie (imdb_id text, title text, year int, rating real, country text)'
)
movies_temp = [
    (m["imdb_id"], m["title"], m["year"], m["rating"], m["country"][0]) for m in movie_actors_data
]
movie_c.executemany(
    'INSERT INTO movie VALUES (?, ?, ?, ?, ?)',
    movies_temp
)
movie.commit()

#### Q4. (5 points) Create the movie_actor table and load data into it.

In [4]:
movie_c.execute(
    'CREATE TABLE movie_actor (imdb_id text, actor text)'
)
movie_actor_temp = []
for m in movie_actors_data:
   movie_actor_temp += [
       (m["imdb_id"], a) for a in m["actors"]
   ] 
movie_c.executemany(
    'INSERT INTO movie_actor VALUES (?, ?)',
    movie_actor_temp
    )
movie.commit()

#### Q5. (10 points) Write an SQL query to find top 10 genres in the US by the number of movies in that genre and print out the results.
```
Top 10 genres:Genre, Movies
Drama,114
Thriller,53
Crime,46
Adventure,41
Mystery,34
Comedy,32
Action,31
Romance,27
Fantasy,21
Sci-Fi,20
```

In [6]:
r = movie_c.execute('SELECT genre, COUNT(genre) from movie_genre JOIN movie ON (movie_genre.imdb_id=movie.imdb_id) WHERE movie.country LIKE "USA" GROUP BY movie_genre.genre ORDER BY Count(movie_genre.genre) DESC')
result = r.fetchall()
print(f"Top 10 genres:Genre, Movies")
for res in result[0:10]:
    print(f"{res[0]},{res[1]}")

Top 10 genres:Genre, Movies
Drama,114
Thriller,53
Crime,46
Adventure,41
Mystery,34
Comedy,32
Action,31
Romance,27
Fantasy,21
Sci-Fi,20


#### Q6. (10 points) Write a SQL query to find the average rating of all movies broken down by year in chronological order.
```
Average rating of movies broken down by year:Year, Rating
1921, 8.3
1922, 8.0
1925, 8.2
1926, 8.3
1927, 8.4
1930, 8.1
1931, 8.55
1934, 8.2
1936, 8.6
1939, 8.233333333333333
1940, 8.333333333333334
1941, 8.4
1942, 8.7
1943, 8.0
1944, 8.25
1946, 8.366666666666665
1948, 8.299999999999999
1949, 8.4
1950, 8.375
1951, 8.1
1952, 8.3
1953, 8.1
1954, 8.42
1955, 8.2
1956, 8.1
1957, 8.466666666666667
1958, 8.4
1959, 8.28
1960, 8.5
1961, 8.25
1962, 8.3
1963, 8.2
1964, 8.3
1965, 8.3
1966, 8.433333333333332
1967, 8.149999999999999
1968, 8.366666666666667
1969, 8.149999999999999
1971, 8.5
1972, 8.649999999999999
1973, 8.166666666666666
1974, 8.75
1975, 8.32
1976, 8.266666666666666
1977, 8.5
1978, 8.2
1979, 8.28
1980, 8.366666666666667
1981, 8.55
1982, 8.200000000000001
1983, 8.3
1984, 8.299999999999999
1985, 8.4
1986, 8.233333333333334
1987, 8.166666666666666
1988, 8.280000000000001
1989, 8.3
1990, 8.8
1991, 8.433333333333332
1992, 8.350000000000001
1993, 8.275
1994, 8.8
1995, 8.35
1996, 8.2
1997, 8.366666666666665
1998, 8.319999999999999
1999, 8.559999999999999
2000, 8.379999999999999
2001, 8.32
2002, 8.48
2003, 8.211111111111112
2004, 8.22
2005, 8.233333333333333
2006, 8.45
2007, 8.157142857142857
2008, 8.4
2009, 8.185714285714285
2010, 8.271428571428572
2011, 8.25
2012, 8.362499999999999
```

In [38]:
r = movie_c.execute('SELECT year, AVG(rating) from movie GROUP BY year ORDER BY year')
result = r.fetchall()
print(f"Average rating of movies broken down by year:Year, Rating")
for (y, s) in result:
    print(f"{y}, {s}")

Average rating of movies broken down by year:Year, Rating
1921, 8.3
1922, 8.0
1925, 8.2
1926, 8.3
1927, 8.4
1930, 8.1
1931, 8.55
1934, 8.2
1936, 8.6
1939, 8.233333333333333
1940, 8.333333333333334
1941, 8.4
1942, 8.7
1943, 8.0
1944, 8.25
1946, 8.366666666666665
1948, 8.299999999999999
1949, 8.4
1950, 8.375
1951, 8.1
1952, 8.3
1953, 8.1
1954, 8.42
1955, 8.2
1956, 8.1
1957, 8.466666666666667
1958, 8.4
1959, 8.28
1960, 8.5
1961, 8.25
1962, 8.3
1963, 8.2
1964, 8.3
1965, 8.3
1966, 8.433333333333332
1967, 8.149999999999999
1968, 8.366666666666667
1969, 8.149999999999999
1971, 8.5
1972, 8.649999999999999
1973, 8.166666666666666
1974, 8.75
1975, 8.32
1976, 8.266666666666666
1977, 8.5
1978, 8.2
1979, 8.28
1980, 8.366666666666667
1981, 8.55
1982, 8.200000000000001
1983, 8.3
1984, 8.299999999999999
1985, 8.4
1986, 8.233333333333334
1987, 8.166666666666666
1988, 8.280000000000001
1989, 8.3
1990, 8.8
1991, 8.433333333333332
1992, 8.350000000000001
1993, 8.275
1994, 8.8
1995, 8.35
1996, 8.2
1997, 8.

#### Q7. (10 points) Write a SQL query to find all Thriller movies from outside the U.S. ordered by decreasing rating, then by increasing year if ratings are the same.

```
Thriller movies:Title, Year, Rating
Léon, 1994, 8.6
M, 1931, 8.5
Das Leben der Anderen, 2006, 8.5
The Third Man, 1949, 8.4
Oldeuboi, 2003, 8.4
Snatch., 2000, 8.3
Lock, Stock and Two Smoking Barrels, 1998, 8.2
El secreto de sus ojos, 2009, 8.2
In the Name of the Father, 1993, 8.1
Amores perros, 2000, 8.1
Mou gaan dou, 2002, 8.1
Salinui chueok, 2003, 8.1
Slumdog Millionaire, 2008, 8.1
```

In [45]:
r = movie_c.execute('SELECT title, year, rating from movie_genre JOIN movie ON (movie_genre.imdb_id=movie.imdb_id) WHERE NOT country LIKE "USA" AND genre LIKE "Thriller" ORDER BY rating DESC, year')
result = r.fetchall()
print(f"Thriller movies:Title, Year, Rating")
for (t, y, r) in result:
    print(f"{t}, {y}, {r}")

Thriller movies:Title, Year, Rating
Léon, 1994, 8.6
M, 1931, 8.5
Das Leben der Anderen, 2006, 8.5
The Third Man, 1949, 8.4
Oldeuboi, 2003, 8.4
Snatch., 2000, 8.3
Lock, Stock and Two Smoking Barrels, 1998, 8.2
El secreto de sus ojos, 2009, 8.2
In the Name of the Father, 1993, 8.1
Amores perros, 2000, 8.1
Mou gaan dou, 2002, 8.1
Salinui chueok, 2003, 8.1
Slumdog Millionaire, 2008, 8.1


#### Q8. (10 points) Write a SQL query to find the top 10 actors based on average movie rating with at least 2 credits in an after year 2000. For each actor, give their name, average rating of the movies they played in, and the number of movies. Sort the result in the descending order based on average movie rating. In case of ties, sort the rows by actor name.

```
In and after year 2000, top 10 highest rated actors based on average movie rating with at least 2 credits : Actor, Movies
Marton Csokas, 8.850000000000001, 2
Noel Appleby, 8.850000000000001, 2
Sean Bean, 8.850000000000001, 2
Bernard Hill, 8.8, 2
Billy Boyd, 8.8, 3
Cate Blanchett, 8.8, 3
John Bach, 8.8, 2
Sean Astin, 8.8, 3
Christopher Lee, 8.75, 2
Joseph Gordon-Levitt, 8.75, 2
```


In [54]:
selection = ''''''
r = movie_c.execute(selection)
result = r.fetchall()
# print(f"Thriller movies:Title, Year, Rating")
# for (t, y, r) in result:
#     print(f"{t}, {y}, {r}")
result

OperationalError: near "AS": syntax error

#### Q9. (10 points) Write a SQL query for finding pairs of actors who co-starred in at least 2 highly rated (rating > 9) movies together. 

The pairs of names must be unique. This means that ‘actor A, actor B’ and ‘actor B, actor A’ are the same pair, so only one of them should appear. For each pair of actors you print out, the two actors must be ordered alphabetically. The pairs are ordered in decreasing number of movies they co-stared in. In case of ties, the rows are ordered by actors’ names. 

>You will need to join the movie_actor table with itself to get this data (in addition to another required join). It is a bit tricky. If you cannot do it with SQL statement, you can also write some Python code that works on the Python data structure that you used to create the movie_actor table. That’ll mean much more lines of code, and if you do it that way, you’ll get 5 points instead of 10 points. You will only get 10 points if you solve it with pure SQL.

```
Pairs of actors who co-stared in 2 or more highly rated movies: Actor A, Actor B, Co-stared Movies
Al Pacino, Diane Keaton, 2
Al Pacino, John Cazale, 2
Al Pacino, Robert Duvall, 2
Al Pacino, Talia Shire, 2
Diane Keaton, John Cazale, 2
Diane Keaton, Robert Duvall, 2
Diane Keaton, Talia Shire, 2
John Cazale, Robert Duvall, 2
John Cazale, Talia Shire, 2
Robert Duvall, Talia Shire, 2
```

## Part 2 (25 points) 

Write a python function that takes two arguments: genre and k 

The function should print out the top k actors who played roles in the highest rated movies (on average) in the provided genre. You should use the sqlite3 database file you created in Part 1. Some example runs of my function are shown below. Your program should produce the same output when provided with the same command line arguments.

`get_top_genre_actors("Drama", 5)`
```
Top 5 actors who played in highest rated Drama movies:
Actor, Average Drama movie rating
Bob Gunton, 9.3
Brian Libby, 9.3
Clancy Brown, 9.3
David Proval, 9.3
Gil Bellows, 9.3
```

`get_top_genre_actors("Action", 10)`
```
Top 10 actors who played in highest rated Action movies:
Actor, Average Action movie rating
Aaron Eckhart, 9.0
Anthony Michael Hall, 9.0
Chin Han, 9.0
Eric Roberts, 9.0
Heath Ledger, 9.0
Maggie Gyllenhaal, 9.0
Monique Gabriela Curnen, 9.0
Nestor Carbonell, 9.0
Ritchie Coster, 9.0
Ron Dean, 9.0
```


`get_top_genre_actors("Comedy", 10)`
```
Top 10 actors who played in highest rated Comedy movies:
Actor, Average Comedy movie rating
Al Ernest Garcia, 8.6
Alba Gaïa Kraghede Bellugi, 8.6
Anne Le Ny, 8.6
Audrey Fleurot, 8.6
Cecil Reynolds, 8.6
Christian Ameri, 8.6
Cyril Mendy, 8.6
Dominique Daguier, 8.6
Dorothée Brière, 8.6
Edward LeSaint, 8.6
```

## What to submit: 
Please perform the following operations and turn in the corresponding files to canvas:
1. Jupyter notebook titled uniqname_si618_hw3.ipynb and the corresponding html page (uniqname_si618_hw3.html) 
through Canvas.
2. The sqlite database file you created. (uniqname_si618_hw3.db)